In [1]:
!pip install -q tfds-nightly tensorflow matplotlib


In [2]:
%load_ext tensorboard

In [22]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from keras.layers import Conv2D, MaxPool2D, ReLU, Concatenate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Dataset Exploration

In [4]:
ds = tfds.load("imagenet_v2", split = 'test')
assert isinstance(ds, tf.data.Dataset)
print(ds)

<PrefetchDataset shapes: {file_name: (), image: (None, None, 3), label: ()}, types: {file_name: tf.string, image: tf.uint8, label: tf.int64}>


In [ ]:
ds = ds.take(10)

for example in ds:  
  # print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print("Image Shape is ",image.shape," and label is ", label.numpy())

In [ ]:
n = 4
fig, axs = plt.subplots(n, 1, figsize = (4, 4*n))
ds = ds.take(n)
i = 0
for example in ds:  
  image = example["image"]
  image = tf.image.resize(image, [64,64])
  image = image/255.0
  axs[i].imshow(image)
  axs[i].set_title("Image")
  label = example["label"]
  label = label.numpy()
  axs[i].text(1.5, 0.5, label,
              horizontalalignment='center',
     verticalalignment='center',
     transform = axs[i].transAxes)
  i=i+1


# Inception Net

In [23]:
def inception_module(x,
                     n_filters_1x1,
                     n_filters_3x3_reduce,
                     n_filters_3x3,
                     n_filters_5x5_reduce,
                     n_filters_5x5,
                     n_filters_pool_1x1):
  
  conv_1x1 = Conv2D(n_filters_1x1, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

  conv_3x3 = Conv2D(n_filters_3x3_reduce, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_3x3 = Conv2D(n_filters_3x3, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

  conv_5x5 = Conv2D(n_filters_5x5_reduce, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_5x5 = Conv2D(n_filters_5x5, (5,5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

  max_pool = MaxPool2D(pool_size=(3, 3), strides=(1,1), padding='same')(x)
  pool_conv_1x1 = Conv2D(n_filters_pool_1x1, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(max_pool)

  output = Concatenate([conv_1x1, conv_3x3, conv_5x5, pool_conv_1x1], axis = 3)

  return output